In [ ]:
# from __future__ import division
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import seaborn as sb
pd.options.display.max_colwidth = 128
pylab.rcParams['figure.figsize'] = 12, 7

In [ ]:
from re import search

In [ ]:
%config 

In [ ]:
import IPython.core.formatters

In [ ]:
IPython.core.formatters

In [ ]:
%config DisplayFormatter

In [ ]:
#%config PlainTextFormatter.pprint = False

In [ ]:
from functools import partial

def test(categorizer, **kw):
    categorize = partial(categorizer, **kw)
    assert categorize('Delta Airlines') == 'Airfare'
    assert categorize('Delta Faucet') == 'Plumbing'
    assert categorize('Comfort Inn') == 'Lodging'
    assert categorize('Drake Hotel') == 'Lodging'
    assert categorize('Sushi Avenue') == 'Restaurant'
    print 'Tests passed'

In [ ]:
def categorize(text):
    if search('Delta', text):
        return 'Airfare'
    if search('Hotel', text):
        return 'Lodging'
    if search('Inn', text):
        return 'Lodging'
    if search('.', text):
        return 'Restaurant'
    return None

print categorize('Comfort Inn')
print categorize('Sushi Avenue')

In [ ]:
import yaml

In [ ]:
rule_text = """\
- /Delta/:
  - /Faucet/:
    - Plumbing
  - Airfare
- Lodging:
  - /Hotel/
  - /Inn/
- //:
  - Restaurant
"""

In [ ]:
rules = yaml.load(rule_text)
rules

In [ ]:
def categorize(text):
    if search('Delta', text):
        return 'Airfare'
    if search('Hotel', text):
        return 'Lodging'
    if search('Inn', text):
        return 'Lodging'
    if search('.', text):
        return 'Restaurant'
    return 'Unknown'

print categorize('Comfort Inn')
print categorize('Sushi Avenue')

# Iteration 1: Interpreter

```
- /Delta/:
  - /Faucet/:
    - Plumbing
  - Airfare
- Lodging:
  - /Hotel/
  - /Inn/
- //:
  - Restaurant
```

In [ ]:
def categorize1(text, rules, category=None):
    for rule in rules:
        subrules = None
        if isinstance(rule, dict):
            rule, subrules = next(rule.iteritems())
        if rule.startswith('/'):
            if not search(rule[1:-1], text):
                continue
        else:
            category = rule
        if subrules:
            category = categorize1(text, subrules, category)
        if category is not None:
            return category

print categorize1('Comfort Inn', rules)
test(categorize1, rules=rules)

# Iteration 2: Compiler

In [ ]:
import ast

tree = ast.parse('2 + 3 * 4')
tree

In [ ]:
# 2 + 3 * 4
# https://docs.python.org/2/library/ast.html#abstract-grammar

tree.body[0]

In [ ]:
print ast.dump(tree)

In [ ]:
import astunparse
print astunparse.dump(tree)

In [ ]:
import astunparse
print astunparse.dump(ast.parse("""
    
    """))

In [ ]:
import astunparse
print astunparse.dump(ast.parse("""
if content == 1:
    return 'foo'
"""))

In [ ]:
from ast import (AST, If, Name, Return, Str, Param, FunctionDef, Interactive,
                 arguments, fix_missing_locations, parse, Num, Expression, Add, BinOp, Mult, Pow, Load, Expr,
                Module, Assign)
from ast import *


tree = Expression(Num(n=42))
fixed = fix_missing_locations(tree)
code = compile(tree, '<luca>', 'eval')
eval(code)

In [ ]:
tree = Expression(BinOp(Num(n=42), Add(), Name(id='n', ctx=Load())))
fixed = fix_missing_locations(tree)
code = compile(tree, '<luca>', 'eval')
eval(code, {}, {'n': 1})

In [ ]:
tree = Interactive([
    Assign([Name(id='m', ctx=Store())], Num(n=2)),
    Expr(BinOp(Num(n=42), Add(), Name(id='m', ctx=Load())))
    ])
fixed = fix_missing_locations(tree)
code = compile(tree, '<luca>', 'single')
eval(code, {}, {'n': 1})

In [ ]:
from dis import dis
dis(code)

```
- /Delta/:
  - /Faucet/:
    - Plumbing
  - Airfare
- Lodging:
  - /Hotel/
  - /Inn/
- //:
  - Restaurant
```

In [ ]:
def compile_test(rule):
    if rule.startswith('/'):
        pattern = Str(rule[1:-1])
        text = Name(id='text', ctx=Load())
        return Compare(pattern, [In()], [text])
    else:
        return Str(rule)

In [ ]:
def compile_rules(rules, category=''):
    return BoolOp(Or(), [compile_rule(r, category) for r in rules])

def compile_rule(rule, category):
    if isinstance(rule, dict):
        rule, subrules = next(rule.iteritems())
        if rule.startswith('/'):
            return BoolOp(And(), [compile_test(rule), compile_rules(subrules, category)])
        return compile_rules(subrules, rule)
    elif rule.startswith('/'):
        return BoolOp(And(), [compile_test(rule), Str(category)])
    return Str(rule)

In [ ]:
tree = compile_rules(rules)
fixed = fix_missing_locations(tree)
code = compile(Expression(tree), '<luca>', 'eval')
categorize2 = lambda text: eval(code, None, {'text': text})

print categorize2('Comfort Inn')
print categorize2('Delta Airlines')
test(categorize2)

In [ ]:
dis(code)

In [ ]:
%timeit categorize1('Sushi Avenue', rules)
%timeit categorize2('Sushi Avenue')

# Iteration 3: Pandas

In [ ]:
# NumPy

x = arange(8)
print x
print x + 1
print x * 10
print x * x

In [ ]:
print sin(x)

In [ ]:
#x = linspace(0, 10, 1000)
plot(x, sin(x) - x / 2.0)

In [ ]:
# Pandas

df = pd.read_csv('transactions.txt', parse_dates=['Date'])
df.head()

In [ ]:
-df.Amount > 500

In [ ]:
df.head()

In [ ]:
c = df.assign(Category=None)
c.head()

In [ ]:
match = c.Description.str.contains('Delta')
c.Category = c.Category[match].fillna('Airfare')
c.head()

In [ ]:
mask = c.Description.str.contains('Inn')
c.Category.mask(mask, 'Lodging')
#c.Category = c.Category[match].fillna('Lodging')
#c.head()

In [ ]:
cc = c.Description.copy()
cc[:] = None
cc.fillna(7)

In [ ]:
def categorize3(descriptions, rules, category=''):
    cats = descriptions.copy()
    cats[:] = None
    for rule in rules:
        cats = cats.fillna(run_rule(descriptions, rule, category))
    return cats

def run_rule(descriptions, rule, category):
    if isinstance(rule, dict):
        rule, subrules = next(rule.iteritems())
        if rule.startswith('/'):
            mask = descriptions.str.contains(rule[1:-1])
            cats = categorize3(descriptions, subrules, category)
            return cats.where(mask, None)
        else:
            return categorize3(descriptions, subrules, rule)
    elif rule.startswith('/'):
        mask = descriptions.str.contains(rule[1:-1])
        cats = descriptions.copy()
        cats[:] = category
        return cats.where(mask, None)
    return rule

print categorize3(df.Description, rules)
#test(categorize1, rules=rules)

In [ ]:
c1 = [categorize1(text, rules) for text in df.Description]
c2 = [categorize2(text) for text in df.Description]
c3 = categorize3(df.Description, rules)
assert c1 == c2
assert (c1 == c3).all()

In [ ]:
%time c1 = [categorize1(text, rules) for text in df.Description]
%time c2 = [categorize2(text) for text in df.Description]
%time c3 = categorize3(df.Description, rules)

In [ ]:
bigdf = pd.concat([df] * 100000)
bigdf.size

In [ ]:
%time c1 = [categorize1(text, rules) for text in bigdf.Description]
%time c2 = [categorize2(text) for text in bigdf.Description]
%time c3 = categorize3(bigdf.Description, rules)